In [33]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np

In [34]:
url = 'https://www.raiffeisen.ru/offices/?active_tab=tab-1'
headers = requests.utils.default_headers()
#headers.update({'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'})
response = requests.get(url)
response.status_code

200

In [93]:
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Chrome('C:/ChromeDriver/chromedriver.exe', chrome_options=options)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


In [99]:
driver.get(url)
switch_buttons = driver.find_elements_by_class_name("b-switch__block")

In [100]:
switch_buttons

[<selenium.webdriver.remote.webelement.WebElement (session="15abbd484b71796cf1f3a51af402f259", element="790d3658-afb5-486d-b25b-ea85d9962fc1")>,
 <selenium.webdriver.remote.webelement.WebElement (session="15abbd484b71796cf1f3a51af402f259", element="a0b6d537-a328-490b-a7b6-2e4721745506")>]

In [101]:
for x in range(len(switch_buttons)):
    if switch_buttons[x].is_displayed():
        driver.execute_script("arguments[0].click();", switch_buttons[x])

In [102]:
import time

In [103]:
more_buttons = driver.find_elements_by_class_name("load-more")
while True:
    buttons_clicked = 0
    for x in range(len(more_buttons)):
        if more_buttons[x].is_displayed():
            driver.execute_script("arguments[0].click();", more_buttons[x])
            time.sleep(1)
            buttons_clicked += 1
            print('pressed\n')
    more_buttons = driver.find_elements_by_class_name("load-more")
    if len(more_buttons) == 0 or buttons_clicked == 0:
        break

pressed

pressed

pressed

pressed

pressed

pressed

pressed

pressed

pressed

pressed

pressed

pressed

pressed

pressed

pressed

pressed

pressed

pressed

pressed

pressed

pressed

pressed

pressed

pressed

pressed

pressed

pressed

pressed

pressed



In [104]:
soup = BeautifulSoup(driver.page_source, 'html5lib')

In [105]:
from collections import defaultdict

In [133]:
dic = defaultdict(dict)
all_opts = set()
for i, data in enumerate(soup.find_all('div', {'class' : 'container e-office-item pb-16 pt-24'})):
    descr = data.find('div', class_='e-office-item__half')
    dic[i]['city'] = descr.a.text.strip().split(',')[0]
    arr = eval(descr.a['data-coords'])
    dic[i]['latitude'] = arr[0]
    dic[i]['longitude'] = arr[1]
    options = data.find('ul', class_='e-office-item__options')
    dic[i]['options'] = []
    for li in options.find_all('li'):
        dic[i]['options'].append(li.text)
        all_opts.add(li.text)
    work_hours = data.find('div', class_='e-office-item__time mb-16').text.split(', ')
    if len(work_hours) == 1:
        dic[i]['work_day_hours'] = work_hours[0][len('пн-пт: '):].strip()
        dic[i]['holiday_hours'] = 'not working'
    elif len(work_hours) == 2:
        dic[i]['work_day_hours'] = work_hours[0][len('пн-пт:'):].strip()
        dic[i]['holiday_hours'] = work_hours[1][len('сб: '):].strip()

In [135]:
all_opts = list(all_opts)

In [138]:
with open('raiffeisen.csv', 'w+', encoding='UTF-8') as f:
    f.write('office_id' + ', ' + 'city' + ', ' + 'latitude' + ', ' + 'longitude' + ', ' + 'work_day_hours' + ', ' + 'holiday_hours')
    f.write(', ' + ', '.join(['opt_{}'.format(i) for i in range(17)]))
    f.write('\n')
    for key in dic:
        f.write(str(key) + ", " + str(dic[key]['city']) + ', ' + str(dic[key]['latitude']))
        f.write(', ' + str(dic[key]['longitude']) + ', ' + dic[key]['work_day_hours'] + ', ' + dic[key]['holiday_hours'])
        for opt in all_opts:
            if opt in dic[key]['options']:
                f.write(', 1')
            else:
                f.write(', 0')
        f.write('\n')

In [136]:
all_opts

['Банкомат 24 часа',
 'Зона обслуживания Premium Banking',
 'Кредитование корпоративного и среднего бизнеса',
 'Детский уголок',
 'Получение кредитной, дебетовой карты или карты, на которую зачислен кредит',
 'Сейфовые ячейки',
 'РКО корпоративного и среднего бизнеса',
 'Зона обслуживания Private Banking',
 'Адаптировано для лиц с ограниченными возможностями',
 'Обмен валюты',
 'Сбор данных в ЕБС',
 'Кредитование малого бизнеса',
 'РКО малого бизнеса',
 'Обслуживание физических лиц',
 'Центр обслуживания ЕСИА',
 'Электронная очередь',
 'Ипотечный центр']

In [141]:
with open('options_dict.csv', 'w+', encoding='UTF-8') as f:
    f.write('code' + ", " + "option_real_name")
    f.write('\n')
    for i, opt in enumerate(all_opts):
        f.write('opt_{}'.format(i) + ', ' + ' '.join(opt.split(', ')))
        f.write('\n')